In [16]:
# inspired by
# https://github.com/fjg00/Facebook-Group-Post-Scraper/blob/main/Facebook%20Group%20Parser.py
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

import argparse
from datetime import datetime
import time
import json

import os
import re
import pandas as pd

In [17]:
# USER DEFINED INPUTS
import Login
CHROME_DRIVER_PATH = './chromedriver' # must download this yourself

# EMAIL, PASSWORD from Login file 
EMAIL = Login.EMAIL
PASSWORD = Login.PASSWORD

# Group number
GROUP_ID = 1379345962387168 #USER INPUT
GROUP_URL = 'https://www.facebook.com/groups/'+str(GROUP_ID)

SEARCH_PROMPT = "" # something to be searched for in the group
SEARCH_PROMPT = SEARCH_PROMPT.replace(" ","%20")

# Copy the group description and put it here to be able to filter it out
GROUP_DESCRIPTION = """-Kamer in Eindhoven-
Kamer in Eindhoven is een platform waar vraag en aanbod (kosteloos) worden samengebracht. Ben je opzoek naar een kamer, huisgenoot of bied je een kamer aan, word dan nu lid van deze facebookgroep.
Je scrollt door de nieuwste kamers en huisgenoten om vervolgens direct te reageren naar de aanbieder. Bij ons geen inschrijfkosten, gewoon studenten onder elkaar.
Kamer in Eindhoven is onderdeel van de Facebook community Zoekt Kamer in… Wij zijn ook actief in Nijmegen, Amsterdam, Delft, Rotterdam, Groningen, Maastricht, Breda, Utrecht, Leiden, Den Haag, Haarlem en Amersfoort.
Sinds 2020 hebben wij een samenwerking met de gratis app MyHospi. Hiermee willen wij het proces voor jullie nog makkelijker maken.
HOE WERKT MYHOSPI VOOR HUIZEN?
1. Plaats de kamer op myHospi via de app.
2. Deel de link naar het huis in je FB bericht en geef aan dat mensen via myHospi (de link) moeten reageren. Met myHospi krijg je direct een compleet beeld van de mensen die reageren op jouw kamer. Geen onpersoonlijke mails maar complete profielen.
3. Swipe en Like!
4. Nodig kandidaten uit en deel de link naar je videocall.""" # User input

In [18]:
def addvalues(dictionary, key, L):
    """Append multiple values to a key in the given dictionary"""
    if key not in dictionary:
        dictionary[key] = list()
    dictionary[key].extend(L)
    return dictionary


In [19]:
# Webdriver options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-notifications')

driver = webdriver.Chrome(CHROME_DRIVER_PATH, options=chrome_options) #USER INPUT
driver.get('https://www.facebook.com/')

WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='_42ft _4jy0 _9xo6 _4jy3 _4jy1 selected _51sy']"))).click()
email = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name = 'email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name = 'pass']")))


email.clear()
password.clear()

email.send_keys(EMAIL) #USER INPUT
password.send_keys(PASSWORD) #USER INPUT

/tmp/ipykernel_38739/2054855826.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(CHROME_DRIVER_PATH, options=chrome_options) #USER INPUT


In [20]:
# Open window, select group
login = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[type = 'submit']"))).click()

#maximize window
driver.maximize_window()

n_scrolls = 2

# Gets FB group with search parameters
driver.get(GROUP_URL+SEARCH_PROMPT)

In [21]:
# Selection criteria
text = ""
comments = ""
z = 1

In [23]:
# Scroll through the page and get all links
all_urls = []
posts_number = 0
posts = []

while len(posts) < 500:
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(5)
    
    a_tags = driver.find_elements(By.TAG_NAME, 'a')
    new_urls = [a.get_attribute('href') for a in a_tags]
    new_comments = [a for a in new_urls if (str(a).startswith(GROUP_URL+'/posts/'))]
    new_posts = set(comment.partition('?comment')[0] for comment in new_comments)
    posts.extend(list(new_posts))
    print ('post count', len(posts))

post count 6
post count 15
post count 28
post count 43
post count 61
post count 81
post count 103
post count 127
post count 155
post count 185
post count 217
post count 251
post count 286
post count 321
post count 357
post count 395
post count 434
post count 475
post count 519


In [24]:
# Grab post and comment data
# TODO make nice df out of it
# TODO grab info about posters and commenters
# TODO grab time data (and time difference between post and comment)


WrapperDict = list() 
texts = []
translatedBools = []
commentCounts = []

for postCount, a in enumerate(posts):
        driver.get(a)
        time.sleep(1)
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        # Relevant classes for text (or translate button)
        translateBtnText = "x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x1n2onr6 x87ps6o x1a2a7pz xt0b8zv"
        translateBtn = soup.find('div', class_ = translateBtnText)
        translatedTextClass = "x193iq5w xeuugli x13faqbe x1vvkbs x10flsy6 x6prxxf xvq8zen xo1l8bm xzsf02u x1yc453h"
        originalTextClass = "x193iq5w xeuugli x13faqbe x1vvkbs x10flsy6 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x41vudc x6prxxf xvq8zen xo1l8bm xzsf02u x1yc453h"

        if (translateBtn is not None) and (soup.find('div', class_ = translateBtnText).text == "See Translation"):
                isTranslated = False
                # Original
                # text = soup.find('span' , class_ = originalTextClass).text #post
                t = soup.find('div', class_ = "x1swvt13 x1l90r2v x1pi30zi x1iorvi4")
                if t is not None:
                        text = t.text #post
                else:
                        # This may result in text="Facebook", could find all of this class and choose one where it doesn't equal Facebook
                        text = soup.find('span' , class_ = originalTextClass).text #post
        elif soup.find('span', class_ = translatedTextClass) is not None:
                # Translated
                text = soup.find('span' , class_ = translatedTextClass).text #post
                isTranslated = True
        elif soup.find('span', class_ = originalTextClass) is not None:
                # Original English
                text = soup.find('span', class_ = originalTextClass).text #post
                isTranslated = False

        # isTranslated = len(soup.find_all('span', class_ = "x193iq5w xeuugli x13faqbe x1vvkbs x10flsy6 x1pg5gke xvq8zen xo1l8bm x1qq9wsj x1yc453h")) > 0
        # TODO instead of length check that it equals expected text ('See original' or 'Rate this translation')

        commentCount = soup.find('span', class_ = "x193iq5w xeuugli x13faqbe x1vvkbs x10flsy6 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x41vudc x6prxxf xvq8zen xo1l8bm xi81zsa").text
        commentCount = int(re.search(r'\d+', commentCount).group())
        
        # TODO create DF/dictionary/smth with all this info
        texts.append(text)
        translatedBools.append(isTranslated)
        commentCounts.append(commentCount)
        if postCount % 5 == 0:
                print (postCount)

        # w = 1
        
        # BranchDict = dict()
        # BranchDict["tag"] = 1
        # BranchDict["patterns"] = 1
        # BranchDict["responses"] = list()
        # L = list()
        
        # for post in actualPosts:
        #     s = post.get_text()
        #     # if (s == "INSERT GROUP DESCRIPTION HERE"): #USER INPUT
        #     if (s == GROUP_DESCRIPTION): #USER INPUT
        #         time.sleep(1)
        #     elif len(s.split()) < 4:
        #         time.sleep(1)
        #     else:
        #         if w == 1: #post
        #             text= text + post.get_text() 
        #             BranchDict["tag"] = SEARCH_PROMPT + str(z) 
        #             BranchDict["patterns"] = text
        #             w = w + text1
        #         elif w > 1 : #comments
        #             comments = post.get_text()
        #             L.append(comments)
        #             comments = ""
        # z = z + 1
        # addvalues(BranchDict, "responses", L)
        # WrapperDict.append(BranchDict)
        # text = ""

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
500
505
510
515


In [25]:
# df = pd.DataFrame([texts, translatedBools, commentCounts], columns=['Text', 'Is_Translated', 'Comment_Count'])
df = pd.DataFrame.from_dict({"Text": texts, "Is_Translated": translatedBools, "Comment_Count": commentCounts, 'URL': list(posts)})

# # Flatten text if necessary
# def flatten(text):
#     if text is list:
#         text = ''.join(text)
#         print (text)
#     return text
# df['Text'] = df['Text'].apply(flatten)
# df.iloc[0]['URL']
df

,Text,Is_Translated,Comment_Count,URL
0,Room available in the Bakkerstraat! (V)A room ...,True,17,https://www.facebook.com/groups/13793459623871...
1,Tijdelijke kamer eindhovenBeste kamerzoekenden...,False,11,https://www.facebook.com/groups/13793459623871...
2,Hey! girls looking for a room wantedA beautif...,True,33,https://www.facebook.com/groups/13793459623871...
3,#offerIn the period 1 December 2022 to 31 May ...,True,5,https://www.facebook.com/groups/13793459623871...
4,"Hello Ladies looking for a room,The coziest ho...",True,10,https://www.facebook.com/groups/13793459623871...
...,...,...,...,...
514,WATCH NIGHT WEDNESDAY OCT 19 (only respond if ...,True,26,https://www.facebook.com/groups/13793459623871...
515,Bedankt voor de aanmeldingen! We hebben genoeg...,False,19,https://www.facebook.com/groups/13793459623871...
516,"Dear lady looking for a room,We are looking fo...",True,17,https://www.facebook.com/groups/13793459623871...
517,"Dear (upcoming) student,We live together with ...",True,2,https://www.facebook.com/groups/13793459623871...


In [26]:
df.describe()

,Comment_Count
count,519.000000
mean,16.206166
std,14.061156
min,1.000000
25%,7.000000
50%,13.000000
75%,26.000000
max,88.000000


In [29]:
# Save it, brother
df.to_csv(path_or_buf='scraped_data.csv', index=False)